In [1]:
import jax
from dataclasses import dataclass
from jax import numpy as jnp
from jax import tree_util as jtu
import numpy as np
import genjax
from genjax import typing
from genjax import adev

console = genjax.pretty()

key = jax.random.PRNGKey(17)

In [2]:
@adev.ADEVProgram
def flip_cond(p):
    b = adev.sample(adev.ADEVPrimBernoulli(), (p,))
    return jax.lax.cond(b, lambda _: 0.0, lambda p: p / 2.0, p)

Exact computation of the above expectation.

$$
E_{p \sim B}[Prog] = (p - p^2) / 2
$$

$$
\nabla_p \{...\} = \frac{1}{2} - p
$$

In [3]:
flip_cond.sample(key, (0.5,))

Array(0.25, dtype=float32)

In [4]:
flip_cond.jvp_estimate(key, (0.5,), (1.0,))

(Array(0.25, dtype=float32), Array(0., dtype=float32))

REINFORCE is:

$$
 \nabla_p E_{b \sim Q(p)}[f(b, p)] = E_{b \sim Q(p)}[\nabla_p f(b, p)] + E_{b \sim Q(p)}[f(b, p) * \nabla_p \log Q(b; p)]
$$